# 1.implement inventory management in mysql

In [1]:
import mysql.connector

# Connect to MySQL database
def connect_to_database():
    try:
        conn = mysql.connector.connect(
            host="127.0.0.1",
            user="root",
            password="password",
            database="Inventory"
        )
        return conn
    except Exception as e:
        print(f"Error: {str(e)}")
        return None

# Add a purchase record to the Purchases table
def add_purchase(conn, product_name, quantity, purchase_date):
    try:
        cursor = conn.cursor()
        sql = "INSERT INTO Purchases (product_name, quantity, purchase_date) VALUES (%s, %s, %s)"
        values = (product_name, quantity, purchase_date)
        cursor.execute(sql, values)
        conn.commit()
        cursor.close()
        print("Purchase record added successfully!")
    except Exception as e:
        print(f"Error: {str(e)}")

# Add a sales record to the Sales table
def add_sale(conn, product_name, quantity, sale_date):
    try:
        cursor = conn.cursor()
        sql = "INSERT INTO Sales (product_name, quantity, sale_date) VALUES (%s, %s, %s)"
        values = (product_name, quantity, sale_date)
        cursor.execute(sql, values)
        conn.commit()
        cursor.close()
        print("Sale record added successfully!")
    except Exception as e:
        print(f"Error: {str(e)}")

# Update the inventory after a purchase or sale
def update_inventory(conn, product_name, quantity_change):
    try:
        cursor = conn.cursor()
        # Check the current quantity in the Inventory table
        cursor.execute("SELECT quantity FROM Inventory WHERE product_name = %s", (product_name,))
        current_quantity = cursor.fetchone()
        if current_quantity:
            current_quantity = current_quantity[0]
        else:
            current_quantity = 0

        # Calculate the new quantity
        new_quantity = current_quantity + quantity_change

        # Update the Inventory table with the new quantity
        cursor.execute("UPDATE Inventory SET quantity = %s WHERE product_name = %s", (new_quantity, product_name))
        conn.commit()
        cursor.close()
        print("Inventory updated successfully!")
    except Exception as e:
        print(f"Error: {str(e)}")

if __name__ == "__main__":
    conn = connect_to_database()
    if conn:
        # Example usage:
        add_purchase(conn, "Product A", 100, "2023-09-15")
        add_sale(conn, "Product A", 50, "2023-09-16")
        update_inventory(conn, "Product A", 50)
        conn.close()

Purchase record added successfully!
Sale record added successfully!
Inventory updated successfully!


# 2

In [4]:

# Connect to the MySQL database
db = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="password",
    database="order_processing"
)

cursor = db.cursor()

# Function to place a new order
def place_order(customer_id, product_id, quantity):
    try:
        # Check if the product is available
        cursor.execute("SELECT quantity_available FROM Products WHERE product_id = %s", (product_id,))
        result = cursor.fetchone()

        if result is not None:
            available_quantity = result[0]
            if available_quantity >= quantity:
                # Calculate the total cost
                cursor.execute("SELECT price FROM Products WHERE product_id = %s", (product_id,))
                result = cursor.fetchone()
                if result is not None:
                    unit_price = result[0]
                    total_price = unit_price * quantity

                    # Insert the order into the Orders and OrderDetails tables
                    cursor.execute("INSERT INTO Orders (customer_id, total_amount) VALUES (%s, %s)", (customer_id, total_price))
                    order_id = cursor.lastrowid
                    cursor.execute("INSERT INTO OrderDetails (order_id, product_id, quantity, unit_price, total_price) VALUES (%s, %s, %s, %s, %s)", (order_id, product_id, quantity, unit_price, total_price))

                    # Update product quantity
                    new_quantity = available_quantity - quantity
                    cursor.execute("UPDATE Products SET quantity_available = %s WHERE product_id = %s", (new_quantity, product_id))

                    db.commit()
                    print("Order placed successfully!")
                else:
                    print("Product price not found.")
            else:
                print("Product is not available in the requested quantity.")
        else:
            print("Product not found.")
    except Exception as e:
        db.rollback()
        print("Error:", e)
    finally:
        cursor.close()
        db.close()

# Example usage
place_order(1, 3, 2)  # Place an order for 2 units of product with ID 101 for customer with ID 1

Order placed successfully!


In [6]:
# Function to generate order reports
def generate_order_report():
    try:
        # Connect to the MySQL database
        db = mysql.connector.connect(
            host="127.0.0.1",
            user="root",
            password="password",
            database="order_processing"
        )
        cursor = db.cursor()

        # Execute the SQL query to retrieve orders
        cursor.execute("SELECT orders.order_id, customers.first_name, customers.last_name, orders.order_date, orders.total_amount FROM orders JOIN customers ON orders.customer_id = customers.customer_id")
        
        # Fetch all orders
        orders = cursor.fetchall()
        
        return orders
    except Exception as e:
        print("Error:", e)
    finally:
        cursor.close()
        db.close()

# Example usage to generate and print an order report
orders = generate_order_report()
if orders:
    for order in orders:
        print("Order ID:", order[0])
        print("Customer:", order[1], order[2])
        print("Order Date:", order[3])
        print("Total Amount:", order[4])
        print()
else:
    print("No orders found.")

Order ID: 1
Customer: John Doe
Order Date: 2023-09-15
Total Amount: 59.97

Order ID: 4
Customer: John Doe
Order Date: None
Total Amount: 19.98

Order ID: 2
Customer: Alice Smith
Order Date: 2023-09-16
Total Amount: 59.98

Order ID: 3
Customer: Bob Johnson
Order Date: 2023-09-17
Total Amount: 19.99



# 3

In [1]:
import mysql.connector

try:
    # Connect to the MySQL database
    db = mysql.connector.connect(
        host="127.0.0.1",
        user="root",
        password="password",
        database="aiml"
    )

    # Create a cursor object to interact with the database
    cursor = db.cursor()

    # Retrieve all records from the people table
    cursor.execute("SELECT * FROM people")

    # Fetch all records
    records = cursor.fetchall()

    # Calculate the total quantity of all records retrieved
    total_quantity = sum(record[2] for record in records)

    # Double the quantity for each record and update the database
    for record in records:
        new_quantity = record[2] * 2
        cursor.execute("UPDATE people SET quantity = %s WHERE id = %s", (new_quantity, record[0]))

    # Commit the changes to the database
    db.commit()
    print("Total quantity before doubling:", total_quantity)
    print("Total quantity after doubling:", total_quantity * 2)
    print("Records updated successfully.")

except mysql.connector.Error as err:
    print("MySQL Error:", err)

except Exception as e:
    print("Error:", e)

finally:
    # Close the cursor and database connection
    if cursor:
        cursor.close()
    if db:
        db.close()

Total quantity before doubling: 0
Total quantity after doubling: 0
Records updated successfully.


# 4

In [2]:
 import mysql.connector

def retrieve_employees_by_department(department_name):
    try:
        # Connect to the MySQL database
        db = mysql.connector.connect(
            host="127.0.0.1",
            user="root",
            password="password",
            database="employee"
        )
        
        cursor = db.cursor()
        
        # Execute a SQL query to retrieve employees in a specific department
        query = "SELECT * FROM Employees WHERE department = %s"
        cursor.execute(query, (department_name,))
        
        # Fetch all records
        employees = cursor.fetchall()
        
        return employees

    except mysql.connector.Error as err:
        print("MySQL Error:", err)
        return None

    finally:
        if cursor:
            cursor.close()
        if db:
            db.close()

# Example usage
department_name = "HR"
employees_in_department = retrieve_employees_by_department(department_name)

if employees_in_department:
    for employee in employees_in_department:
        print("Employee ID:", employee[0])
        print("Name:", employee[1])
        print("Salary:", employee[2])
        print("Department:", employee[3])
        print("Hire Date:", employee[4])
        print()
else:
    print("No employees found in the department.")


 

Employee ID: 1
Name: hema
Salary: 55000.00
Department: HR
Hire Date: 2023-09-15



In [3]:
def update_employee_salary(employee_id, new_salary):
    try:
        # Connect to the MySQL database
        db = mysql.connector.connect(
            host="127.0.0.1",
            user="root",
            password="password",
            database="employee"
        )
        
        cursor = db.cursor()
        
        # Execute a SQL query to update the employee's salary
        query = "UPDATE Employees SET salary = %s WHERE employee_id = %s"
        cursor.execute(query, (new_salary, employee_id))
        
        # Commit the changes to the database
        db.commit()
        print("Employee's salary updated successfully.")

    except mysql.connector.Error as err:
        print("MySQL Error:", err)

    finally:
        if cursor:
            cursor.close()
        if db:
            db.close()

# Example usage to update an employee's salary
employee_id_to_update = 1
new_salary_value = 60000.00
update_employee_salary(employee_id_to_update, new_salary_value)

Employee's salary updated successfully.
